In [28]:
## импорты 
import re,codecs
import json
from lxml import etree
from os import walk
from pymystem3 import Mystem
m = Mystem()

In [29]:
## определяем input -- output 
infolder = '../../tei' # ## папка с TEI/XML #  'testtei' 
results = 'results/speechcorpus'
#bycharacter = results+'/forStylo/bycharacter' #'/bycharacter'
bysexandgenre = results+'/bysexandgenre'
#outfilename = 'stats_per_play_with_dirtext_and_more_stats_07_02_2018.csv'
#outfile = codecs.open (results+outfilename, 'w', 'utf-8')
ns = {'tei': 'http://www.tei-c.org/ns/1.0','xml':'http://www.w3.org/XML/1998/namespace'}
#header = ['play name','written','print', 'number of stages','len_stages_words','len_speeches_words','total_number_of_verbs','verbs set','number of unique verbs','stages_to_speeches_ratio','verb_diversity(uniqueverbs/totalverbs)','all_stage_texts']

In [30]:
def getgenre (root):
    term = root.find('.//tei:textClass/tei:keywords/tei:term', ns)
    if term != None:
        if 'subtype' in term.attrib:
            return term.attrib['subtype']
        else:
            return 'unknown_genre'
    else:
        return 'unknown_genre'
def getnameandauthorandgenre (root):
    name = (re.sub('\r|\n','',root.find('.//tei:titleStmt/tei:title', ns).text)) 
    author = (re.sub('\r|\n|,','',root.find('.//tei:titleStmt/tei:author', ns).text)) 
    genre = getgenre (root)
    return name, author, genre


def getwhen(date):
    if 'when' in date.attrib:
        return date.attrib ['when']
    else:
        return 'not specified in tei'
    
def getdates (root):
    written = getwhen(root.find('.//tei:bibl/tei:bibl/tei:date[@type="written"]', ns))
    printdate = getwhen(root.find('.//tei:bibl/tei:bibl/tei:date[@type="print"]', ns))
    return (written, printdate)
    #return (.text)

def checknotmult(speaker):
    if re.search (' #', speaker) != None:
        return False
    else:
        return True    
    
def getsex (speaker, root):
    #print (speaker)
    thisperson = root.find('.//tei:listPerson/tei:person[@xml:id="'+speaker+'"]',ns) #', ns)
    if 'sex' in thisperson.attrib:
        return (thisperson.attrib['sex'])
    else:
        return 'undefined'
    #return root.find('.//tei:listPerson/tei:person[@id="'+speaker+'"]/@sex', ns)

def getspeakerandsex(somesp, root):
    speaker = 'unknown'
    sex = 'undefined'
    notmult = False
    if 'who' in somesp.attrib:
        speaker = somesp.attrib['who']
        notmult = checknotmult(speaker)
        speaker = re.sub('#','',speaker)
        if notmult:
            sex = getsex (speaker, root) 
    return speaker, sex, notmult

def getverbs(text):
    verbs = set()
    verbcounter = 0
    analysis = m.analyze(text) #json.dumps(m.analyze(text)) # , ensure_ascii=False, encoding='utf-8'
    for word in analysis:
        #print (word)
        if 'analysis' in word:
            if len (word['analysis'])>0:
                word_analysis = word['analysis'][0]
            #print (word_analysis)
                lemma = word_analysis['lex']
                gram = word_analysis ['gr'].split (',')
                if gram[0] == 'V':
                    verbcounter+=1
                    #print (text)
                    #print (lemma)
                    verbs.add(lemma)
    return verbs, verbcounter

def settostring (someset):
    string =''
    for item in someset:
        string+= item+','
    return string



In [31]:
import random
def parse_xml(path, filename):
    fullpath = path+'/'+filename
    root = etree.parse(fullpath)
    name, author, genre = getnameandauthorandgenre (root)
    #written, printdate = getdates (root)
    #thisplay = Play (filename,name,written, printdate,0,0,0,0,set(),0,'')
    
    ## в этом цикле мы обходим все <sp> в XML-документе в
    for sp in root.iterfind ('.//tei:sp', ns):
        speaker, speakersex, notmult = getspeakerandsex (sp, root)
        #counter+=1
        if notmult: 
            ##[not(ancestor::tei:stage)][ancestor::tei:p]|//text()[not(ancestor::tei:stage)][ancestor::tei:l] 
            speechtext = ' '.join(sp.xpath('.//tei:p/text()|.//tei:l/text()',namespaces={'tei': 'http://www.tei-c.org/ns/1.0'}))+' '
            #print (speechtext)
            randombinary = str (random.randint(1,2)) ## for stylo -- requires at least two specimen
            speechcode = '_'.join([speakersex,genre,randombinary])
            genderandgenrefile = codecs.open(bysexandgenre+'/'+speechcode+'.txt','a','utf-8')
            # = codecs.open(bysex+'/'+speakersex+'.txt','a','utf-8')
            genderandgenrefile.write(speechtext)
            genderandgenrefile.close()

In [32]:
for path, dirs, filenames in walk (infolder):
    for filename in filenames:
        if '.xml' in filename:
            print (filename)
            play = parse_xml (path,filename)

print ('DONE PARSING')

pushkin-stseny-iz-rytsarskih-vremen.xml
turgenev-holostjak.xml
gogol-zhenitba.xml
krylov-sonnyj-poroshok-ili-pohischennaja-krestjanka.xml
blok-neznakomka.xml
gogol-lakeiskaja.xml
bulgakov-dni-turbinyh.xml
pushkin-pir-vo-vremja-chumy.xml
sukhovo-kobylin-smert-tarelkina.xml
gumilyov-ditja-allaha.xml
pushkin-boris-godunov.xml
prutkov-blondy.xml
shakhovskoy-ne-lubo-ne-slushai.xml
blok-korol-na-ploschadi.xml
gumilyov-don-zhuan-v-egipte.xml
prutkov-chereposlov-sirech-frenolog.xml
gogol-revizor.xml
gumilyov-gondla.xml
chekhov-leshii.xml
sumarokov-horev.xml
gorky-egor-bulychov-i-drugie.xml
turgenev-vecher-v-sorrente.xml
pushkin-kamenniy-gost.xml
chekhov-jubilej.xml
shakhovskoy-pustodomy.xml
krylov-podschipa-ili-trumf.xml
sukhovo-kobylin-svadba-krechinskogo.xml
lomonosov-tamira-i-selim.xml
chekhov-chaika.xml
gorky-na-dne.xml
chekhov-medved.xml
sumarokov-semira.xml
turgenev-nahlebnik.xml
sumarokov-dimitrij-samozvanets.xml
naydyonov-deti-vanjushina.xml
chekhov-vishnevyi-sad.xml
ostrovsky-besprida